In [17]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [19]:
df = pd.read_csv('survey.csv')
print("Dataset loaded successfully.")
df

Dataset loaded successfully.


,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,2015-09-12 11:17:21,26,male,United Kingdom,NaN,No,No,Yes,NaN,26-100,...,Somewhat easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN
1255,2015-09-26 01:07:35,32,Male,United States,IL,No,Yes,Yes,Often,26-100,...,Somewhat difficult,No,No,Some of them,Yes,No,No,Yes,No,NaN
1256,2015-11-07 12:36:58,34,male,United States,CA,No,Yes,Yes,Sometimes,More than 1000,...,Somewhat difficult,Yes,Yes,No,No,No,No,No,No,NaN
1257,2015-11-30 21:25:06,46,f,United States,NC,No,No,No,NaN,100-500,...,Don't know,Yes,No,No,No,No,No,No,No,NaN


In [20]:
print("Shape of the dataset (rows, columns):", df.shape)
print("\nInformation about the dataset:")
df.info()

print("\nDescriptive statistics for numerical columns:")
print(df.describe())

print("\nMissing values per column:")
print(df.isnull().sum())

for col in df.select_dtypes(include='object').columns:
    print(f"\nUnique values in '{col}':")
    print(df[col].unique()[:20]) # Print first 20 unique values
    print(f"Number of unique values: {df[col].nunique()}")

Shape of the dataset (rows, columns): (1259, 27)

Information about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1259 non-null   object
 1   Age                        1259 non-null   int64 
 2   Gender                     1259 non-null   object
 3   Country                    1259 non-null   object
 4   state                      744 non-null    object
 5   self_employed              1241 non-null   object
 6   family_history             1259 non-null   object
 7   treatment                  1259 non-null   object
 8   work_interfere             995 non-null    object
 9   no_employees               1259 non-null   object
 10  remote_work                1259 non-null   object
 11  tech_company               1259 non-null   object
 12  benefits                   1259 non-n

In [21]:

# Handle 'Age':

# Some values might be outliers or invalid (e.g., negative, very high).
print("\nUnique 'Age' values before cleaning:", sorted(df['Age'].unique()))

df = df[(df['Age'] >= 18) & (df['Age'] <= 75)]

# Fill any potential NaN values if they arise from other operations, though 'Age' seems complete in the sample
df['Age'].fillna(df['Age'].median(), inplace=True)


# Handle 'Gender':
# Clean inconsistent entries.
print("\nUnique 'Gender' values before cleaning:", df['Gender'].unique())
df['Gender'] = df['Gender'].str.lower()
gender_map = {
    'male': 'Male', 'm': 'Male', 'male-ish': 'Male', 'maile': 'Male', 'mal': 'Male', 'male (cis)': 'Male', 'make': 'Male', 'cis male': 'Male',
    'female': 'Female', 'f': 'Female', 'woman': 'Female', 'femake': 'Female', 'female (cis)': 'Female', 'cis female': 'Female',
    # Grouping non-binary/other for simplicity. You might choose a different strategy.
    'non-binary': 'Other', 'genderqueer': 'Other', 'agender': 'Other', 'androgyne': 'Other', 'queer': 'Other', 'fluid': 'Other', 'enby': 'Other', 'trans-female': 'Other', 'trans woman': 'Other', 'male leaning androgynous': 'Other', 'neuter': 'Other', 'queer/she/they': 'Other', 'something else': 'Other', 'ostensibly male, unsure what that means': 'Other', 'nah': 'Other', 'guy (-ish) ^_^': 'Other', 'a little about you': 'Other', 'p': 'Other'
}

#For simplicity, we will map common ones and others to 'Other'.
df['Gender'] = df['Gender'].apply(lambda x: gender_map.get(x, 'Other'))
df = df[df['Gender'].isin(['Male', 'Female', 'Other'])] # Keep only these categories
print("Unique 'Gender' values after cleaning:", df['Gender'].unique())


# 4. Handle 'Country' and 'state':
df['state'].fillna('Not_US_Or_Unknown', inplace=True)
print("\nUnique 'state' values after cleaning 'NA':", df['state'].unique()[:10])


# 5. Handle 'self_employed':
# Contains 'NA'. Decide how to treat 'NA' - e.g., as a separate category, impute, or map to 'No'/'Yes' if context allows.

df['self_employed'].fillna(df['self_employed'].mode()[0] if not df['self_employed'].mode().empty else 'Unknown', inplace=True)
print("\nUnique 'self_employed' values after cleaning 'NA':", df['self_employed'].unique())


# 6. Handle 'work_interfere':
df['work_interfere'].fillna('Unknown', inplace=True) # Example: treat NA as 'Unknown'
print("\nUnique 'work_interfere' values after cleaning 'NA':", df['work_interfere'].unique())

# 7. Handle 'comments':
# This is text data. For your current supervised learning model, this might be dropped unless you plan to do NLP feature extraction.
df.drop('comments', axis=1, inplace=True)
df.drop('Timestamp', axis=1, inplace=True) # Dropping timestamp as per earlier note

# 8. Handle 'Don't know' and 'Not sure' values in other categorical columns
cols_with_dont_know = ['benefits', 'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave', 'mental_vs_physical']
for col in cols_with_dont_know:
    if col in df.columns:
        df[col].replace("Don't know", 'Unknown', inplace=True)
        df[col].replace("Not sure", 'Unknown', inplace=True)
        print(f"Cleaned 'Don't know'/'Not sure' in {col}: {df[col].unique()[:10]}")


# 9. Check for remaining NaN values after initial cleaning
print("\nMissing values after initial cleaning steps:")
print(df.isnull().sum())

# Fill any remaining NaNs for categorical with mode and numerical with median
for col in df.columns:
    if df[col].isnull().any():
        if df[col].dtype == 'object':
            df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 'Unknown', inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)

print("\nMissing values after final fill (should be all zeros):")
print(df.isnull().sum())

print("\nData after initial cleaning:")
df.head()


Unique 'Age' values before cleaning: [-1726, -29, -1, 5, 8, 11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 60, 61, 62, 65, 72, 329, 99999999999]

Unique 'Gender' values before cleaning: ['Female' 'M' 'Male' 'male' 'female' 'm' 'Male-ish' 'maile' 'Trans-female'
 'Cis Female' 'F' 'something kinda male?' 'Cis Male' 'Woman' 'f' 'Mal'
 'Male (CIS)' 'queer/she/they' 'non-binary' 'Femake' 'woman' 'Make' 'Nah'
 'Enby' 'fluid' 'Genderqueer' 'Female ' 'Androgyne' 'Agender'
 'cis-female/femme' 'Guy (-ish) ^_^' 'male leaning androgynous' 'Male '
 'Man' 'Trans woman' 'msle' 'Neuter' 'Female (trans)' 'queer'
 'Female (cis)' 'Mail' 'cis male' 'Malr' 'femail' 'Cis Man'
 'ostensibly male, unsure what that really means']
Unique 'Gender' values after cleaning: ['Female' 'Male' 'Other']

Unique 'state' values after cleaning 'NA': ['IL' 'IN' 'Not_US_Or_Unknown' 'TX' 'TN' 'MI' 'OH' 'CA' 'CT'

,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,...,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,37,Female,United States,IL,No,No,Yes,Often,6-25,No,...,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,44,Male,United States,IN,No,No,No,Rarely,More than 1000,No,...,Unknown,Unknown,Maybe,No,No,No,No,No,Unknown,No
2,32,Male,Canada,Not_US_Or_Unknown,No,No,No,Rarely,6-25,No,...,Unknown,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No
3,31,Male,United Kingdom,Not_US_Or_Unknown,No,Yes,Yes,Often,26-100,No,...,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,31,Male,United States,TX,No,No,No,Never,100-500,Yes,...,Unknown,Unknown,No,No,Some of them,Yes,Yes,Yes,Unknown,No


In [22]:
# Cell 6: Feature Engineering and Encoding

# 1. Define Target Variable: 'treatment'
# "Whether the individual has sought or needs mental health treatment (binary classification)"
# Map 'Yes' to 1 and 'No' to 0.
if 'treatment' in df.columns:
    df['treatment_target'] = df['treatment'].apply(lambda x: 1 if x == 'Yes' else 0)
    print("\nTarget variable 'treatment_target' created.")
    print(df[['treatment', 'treatment_target']].head())
    # We can drop the original 'treatment' column now
    # df.drop('treatment', axis=1, inplace=True) # Keep it for now to verify, drop before modeling
else:
    print("\nERROR: 'treatment' column not found to create target variable!")


# 2. Identify Feature Types
# Demographics (age, gender, location - country/state)
# Workplace factors (remote_work, tech_company, benefits, care_options, wellness_program, seek_help, anonymity, leave, mental_health_consequence, phys_health_consequence, coworkers, supervisor, mental_health_interview, phys_health_interview, mental_vs_physical, no_employees, self_employed, work_interfere)
# Personal history (family_history, obs_consequence) - 'treatment' is now target

numerical_features = ['Age']
# Categorical features will be all object types except the original target if we keep it.
categorical_features = [col for col in df.select_dtypes(include='object').columns if col != 'treatment'] # Exclude original target
if 'treatment_target' in df.columns: # Ensure target is not in features
    if 'treatment' in categorical_features:
      categorical_features.remove('treatment')

print("\nNumerical Features:", numerical_features)
print("Categorical Features:", categorical_features)


# 3. Preprocessing Pipelines
# For numerical features: Standard Scaling
numerical_transformer = StandardScaler()

# For categorical features: One-Hot Encoding
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse_output=False for dense array

# Create a preprocessor object using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # In case some columns are not specified, good for debugging
)

# Apply the transformations (this is often done within a pipeline with the model)
# For demonstration, let's fit and transform the features (X)
X = df.drop(['treatment_target', 'treatment'], axis=1, errors='ignore') # Drop target and original treatment
y = df['treatment_target'] if 'treatment_target' in df.columns else pd.Series()


# Ensure all feature names are strings (ColumnTransformer needs this)
X.columns = X.columns.astype(str)

# Fit preprocessor and transform X
try:
    X_processed_array = preprocessor.fit_transform(X)
    # Get feature names after one-hot encoding
    feature_names_out = preprocessor.get_feature_names_out()
    X_processed = pd.DataFrame(X_processed_array, columns=feature_names_out)
    print("\nShape of processed features X_processed:", X_processed.shape)
    print("Head of processed features X_processed:")
    print(X_processed.head())
except Exception as e:
    print(f"Error during preprocessing: {e}")
    print("Make sure all feature lists and dataframe columns align.")
    X_processed = X # Fallback for now if error


Target variable 'treatment_target' created.
  treatment  treatment_target
0       Yes                 1
1        No                 0
2        No                 0
3       Yes                 1
4        No                 0

Numerical Features: ['Age']
Categorical Features: ['Gender', 'Country', 'state', 'self_employed', 'family_history', 'work_interfere', 'no_employees', 'remote_work', 'tech_company', 'benefits', 'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave', 'mental_health_consequence', 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview', 'mental_vs_physical', 'obs_consequence']

Shape of processed features X_processed: (1251, 158)
Head of processed features X_processed:
   num__Age  cat__Gender_Female  cat__Gender_Male  cat__Gender_Other  \
0  0.675775                 1.0               0.0                0.0   
1  1.636606                 0.0               1.0                0.0   
2 -0.010533             

In [ ]:
# Modeling

if 'treatment_target' not in df.columns:
    print("Target variable 'treatment_target' is missing. Cannot proceed with modeling.")
else:
    X = df.drop(['treatment_target', 'treatment'], axis=1, errors='ignore')
    y = df['treatment_target']
    X.columns = X.columns.astype(str) # Ensure column names are strings for transformers

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y) # Stratify for imbalanced classes

    print("X_train shape:", X_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_train distribution:\n", y_train.value_counts(normalize=True))
    print("y_test distribution:\n", y_test.value_counts(normalize=True))

    # Define models
    models = {
        "Logistic Regression": LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced'), # class_weight for imbalance
        "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced'),
        "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', scale_pos_weight= (y_train.value_counts()[0] / y_train.value_counts()[1] if len(y_train.value_counts()) > 1 and y_train.value_counts()[1] !=0 else 1) ) # scale_pos_weight for imbalance
    }

    results = {}

    for model_name, model in models.items():
        print(f"\n--- Training and Evaluating: {model_name} ---")
        # Create a full pipeline with preprocessor and model
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', model)])

        # Train the model
        try:
            pipeline.fit(X_train, y_train)

            # Make predictions
            y_pred_train = pipeline.predict(X_train)
            y_pred_test = pipeline.predict(X_test)

            # Evaluate the model
            accuracy_train = accuracy_score(y_train, y_pred_train)
            accuracy_test = accuracy_score(y_test, y_pred_test)
            precision_test = precision_score(y_test, y_pred_test, zero_division=0)
            recall_test = recall_score(y_test, y_pred_test, zero_division=0)
            f1_test = f1_score(y_test, y_pred_test, zero_division=0)

            results[model_name] = {
                "Accuracy (Train)": accuracy_train,
                "Accuracy (Test)": accuracy_test,
                "Precision (Test)": precision_test,
                "Recall (Test)": recall_test,
                "F1-score (Test)": f1_test
            }

            print(f"Results for {model_name}:")
            print(f"  Accuracy (Train): {accuracy_train:.4f}")
            print(f"  Accuracy (Test): {accuracy_test:.4f}")
            print(f"  Precision (Test): {precision_test:.4f}")
            print(f"  Recall (Test): {recall_test:.4f} (High recall preferred)")
            print(f"  F1-score (Test): {f1_test:.4f}")

            print("\nClassification Report (Test Set):")
            print(classification_report(y_test, y_pred_test, zero_division=0))

            print("\nConfusion Matrix (Test Set):")
            cm = confusion_matrix(y_test, y_pred_test)
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.title(f'Confusion Matrix - {model_name}')
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            plt.show()

            # Feature Importance (for tree-based models)
            if hasattr(model, 'feature_importances_'):
                try:
                    # Get feature names from the preprocessor step
                    feature_names = preprocessor.get_feature_names_out()
                    importances = model.feature_importances_
                    feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
                    feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False).head(15) # Top 15

                    plt.figure(figsize=(10, 8))
                    sns.barplot(x='importance', y='feature', data=feature_importance_df)
                    plt.title(f'Feature Importances - {model_name}')
                    plt.show()
                except Exception as e_fi:
                    print(f"Could not plot feature importances for {model_name}: {e_fi}")
        except ValueError as ve:
            print(f"ValueError during model training for {model_name}: {ve}")
            print("This might be due to very few samples after splitting or issues with data types/categories.")
        except Exception as e:
            print(f"An unexpected error occurred with {model_name}: {e}")


    # Display results in a table
    results_df = pd.DataFrame(results).T
    print("\n--- Summary of Model Results ---")
    print(results_df)

In [24]:
#Testing and Evaluation

# 1. Cross-Validation (Example with one model)
if 'Logistic Regression' in models and 'treatment_target' in df.columns:
    print("\n--- Cross-Validation for Logistic Regression ---")
    lr_pipeline_cv = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('classifier', LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced'))])
    try:
        cv_scores_recall = cross_val_score(lr_pipeline_cv, X, y, cv=5, scoring='recall') # 5-fold CV, focusing on recall
        cv_scores_f1 = cross_val_score(lr_pipeline_cv, X, y, cv=5, scoring='f1')

        print(f"Cross-Validation Recall Scores: {cv_scores_recall}")
        print(f"Mean CV Recall: {cv_scores_recall.mean():.4f} (+/- {cv_scores_recall.std():.4f})")
        print(f"Cross-Validation F1 Scores: {cv_scores_f1}")
        print(f"Mean CV F1: {cv_scores_f1.mean():.4f} (+/- {cv_scores_f1.std():.4f})")
    except ValueError as ve:
        print(f"ValueError during cross-validation: {ve}. Potentially too few samples in a split for stratify or a class.")
    except Exception as e:
        print(f"Error during cross-validation: {e}")


# 2. Hyperparameter Tuning (Example with RandomForest - this can be time-consuming)
if 'Random Forest' in models and 'treatment_target' in df.columns:
    print("\n--- Hyperparameter Tuning for Random Forest (Example) ---")
    rf_pipeline_gs = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))])

    # Reduced parameter grid for quicker demonstration
    param_grid_rf = {
        'classifier__n_estimators': [50, 100], # Number of trees
        'classifier__max_depth': [None, 10],    # Max depth of trees
        # 'classifier__min_samples_split': [2, 5],
        # 'classifier__min_samples_leaf': [1, 2]
    }
    try:
        grid_search_rf = GridSearchCV(rf_pipeline_gs, param_grid_rf, cv=2, scoring='recall', verbose=1, n_jobs=-1) # Using recall
        grid_search_rf.fit(X_train, y_train)

        print("Best Parameters for Random Forest (Recall):", grid_search_rf.best_params_)
        print("Best CV Recall Score for Random Forest:", grid_search_rf.best_score_)

        # Evaluate the best model from GridSearchCV on the test set
        best_rf_model = grid_search_rf.best_estimator_
        y_pred_test_best_rf = best_rf_model.predict(X_test)
        print("\nRandom Forest (Best Model from GridSearch) - Test Set Performance:")
        print(f"  Recall (Test): {recall_score(y_test, y_pred_test_best_rf, zero_division=0):.4f}")
        print(f"  F1-score (Test): {f1_score(y_test, y_pred_test_best_rf, zero_division=0):.4f}")
        print(classification_report(y_test, y_pred_test_best_rf, zero_division=0))
    except ValueError as ve:
         print(f"ValueError during GridSearchCV: {ve}. Check data splits and class distributions.")
    except Exception as e:
        print(f"Error during GridSearchCV: {e}")


--- Cross-Validation for Logistic Regression ---
Cross-Validation Recall Scores: [0.8503937  0.87401575 0.8968254  0.9047619  0.88888889]
Mean CV Recall: 0.8830 (+/- 0.0192)
Cross-Validation F1 Scores: [0.81509434 0.85384615 0.84328358 0.85714286 0.81454545]
Mean CV F1: 0.8368 (+/- 0.0185)

--- Hyperparameter Tuning for Random Forest (Example) ---
Fitting 2 folds for each of 4 candidates, totalling 8 fits
Best Parameters for Random Forest (Recall): {'classifier__max_depth': 10, 'classifier__n_estimators': 50}
Best CV Recall Score for Random Forest: 0.8987341772151899

Random Forest (Best Model from GridSearch) - Test Set Performance:
  Recall (Test): 0.8861
  F1-score (Test): 0.8309
              precision    recall  f1-score   support

           0       0.87      0.75      0.80       155
           1       0.78      0.89      0.83       158

    accuracy                           0.82       313
   macro avg       0.82      0.82      0.82       313
weighted avg       0.82      0.82  